# Exercise 2 - Trees
Vi ska använda tre modeller för att avgöra om en person överlever Titanic eller inte
Uppgift:
1. Använd din preparerade datafil från föregånede Exercise 1, alternativt använd den förpreparerade titanic.csv-filen.
2. Välj ut och skapa dina egna features
3. Dela upp ditt dataset i träning och test

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('titanic.csv')

## Decision Tree
1. Använd ett decision tree, en random forest och XGBoost för att skapa predictions
2. Plotta trädet för att se hur det ser ut. Går det att förstå något?

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
model = DecisionTreeClassifier() # optimize! Can we do it better?

## Random Forest
1. Använd en random forest för att skapa predictions.
2. Är denna bättre än vårt Decision Tree?
3. En RandomForest-modell består av flera DecisionTrees. Dessa går att få fram genom model_forest.estimators_. Plotta de 5 första träden i vår RandomForest för att se hur det ser ut. Går det att förstå något?

In [9]:
from sklearn.ensemble import RandomForestClassifier
model_forest = RandomForestClassifier() # optimize! Can we do it better?

/var/folders/t3/c9yl4b8s3fd5vzr9v1gp94ym0000gp/T/ipykernel_26632/1013424532.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_forest.fit(X_train, y_train)


## XGBoost
1. Använd en XGBoostClassifier för att skapa predictions.
2. Är denna bättre än vårt Decision Tree och RandomForest?
3. Använd from xgboost import plot_tree för att plotta vår XGBoostClassifier för att se hur det ser ut. Går det att förstå något?

In [12]:
import xgboost as xgb
from xgboost import plot_tree
model_XGB = xgb.XGBClassifier() # optimize! Can we do it better?